In [6]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression

In [7]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [90]:
model.coef_

array([-9.42292260e-03, -2.19168243e-03, -2.89661059e-03,  2.48669564e-03,
       -2.32594358e-03,  5.28586910e-03,  4.13043399e-02, -5.29134900e-04,
       -3.26574510e-03, -3.97754539e-03, -2.66492257e-03,  1.25701467e-05,
        2.63392473e-05])

In [63]:
df = pd.read_sql(
    """
    select date, ticker, acc, agr, beta, bm, ep, gma, idiovol,
    lev, mom12m, mom1m, mve, operprof, roeq, ret
    from data
    order by date, ticker
    """,
    conn
  )
df = df.dropna()
df = df.set_index(["date", "ticker"])

In [73]:
later = df.index.get_level_values("date")>="2010-01"

train = df[~later]
test = df[later]

In [67]:
predictors = [
  "acc", "agr", "beta", "bm", "ep", "gma", "idiovol",
  "lev", "mom12m", "mom1m", "mve", "operprof", "roeq"
]

Xtrain = train[predictors]
ytrain = train["ret"]

model = LinearRegression()
_ = model.fit(Xtrain, ytrain)

In [ ]:
print(model.intercept_)

coefs = pd.Series(model.coef_, index=predictors)
coefs

In [75]:
Xtest = test[predictors]

ypredict = model.predict(Xtest)
ypredict = pd.Series(ypredict, index=test.index)

In [76]:
cut = lambda x: pd.qcut(x, 5, labels=range(1, 6))
quintiles = ypredict.groupby("date", group_keys=False).apply(cut)
quintiles.name = "quintile"

In [83]:
test = test.join(quintiles)

In [86]:

rets = test.reset_index().groupby(["date", "quintile"]).ret.mean()
rets = rets.unstack()
rets.head()

quintile,1,2,3,4,5
date,,,,,
2010-01,-0.027476,-0.027824,-0.011851,0.006101,0.041327
2010-02,0.044193,0.040953,0.036378,0.040231,0.033047
2010-03,0.076036,0.071594,0.081333,0.078276,0.098675
2010-04,0.053192,0.071345,0.071672,0.080754,0.123110
2010-05,-0.088657,-0.075424,-0.070076,-0.079732,-0.089493


In [87]:
rets.mean()

quintile
1    0.011785
2    0.012492
3    0.012492
4    0.013040
5    0.014558
dtype: float64